In [1]:
import scipy.io
import numpy as np
import random
import os

In [2]:
person_nums = [1,2,4,5,6,8,9,10,11,12,13,17,19,21,22,25,26,27,28,29]
## Build the baseline model for emotion recognition with dropout layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, SimpleRNN, LSTM, Conv2D, Flatten, MaxPooling2D, GRU, AveragePooling2D, Dropout, Lambda
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from feature_emotion import feature_extract, split_data, label_unique_tuples, split_data_unknown, split_data_few_shot
gts, sensor_nums, walk_nums, trace_nums, people_nums, spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features = feature_extract(person_nums)

walk_nums_all = np.squeeze(walk_nums)
trace_nums_all = np.squeeze(trace_nums)
people_nums_all = np.squeeze(people_nums)
ra = 0.1
test_person_id = [6]
flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

2023-12-04 19:00:30.619547: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/lib/python3.11/site-packages/h5py/__init__.py:36: UserWarning: h5py is running against HDF5 1.14.2 when it was built against 1.14.1, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


35165
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_30.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


2023-12-04 19:01:53.600989: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 19:01:53.612343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-04 19:01:53.612582: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Test MAE:
1.324290821235424
[1.10579594 1.44051578]
1.273155861430698


In [4]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_31.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:02:11.035666: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_54/lstm_cell_54/recurrent_kernel/Assign' id:12327 op device:{requested: '', assigned: ''} def:{{{node lstm_54/lstm_cell_54/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_54/lstm_cell_54/recurrent_kernel, lstm_54/lstm_cell_54/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:02:12.355176: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_63/lstm_cell_63/kernel/m/Assign' id:16589 op device:{requested: '', assigned: ''} def:{{{node lstm_63/lstm_cell_63/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_63/lstm_cell_63/kernel/m, lstm_63/lstm_cell_63/kerne

Test MAE:
1.3269873387413607
[1.1163885  1.46075216]
1.28857033081304


In [5]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_32.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:02:27.047123: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_110/lstm_cell_110/recurrent_kernel/Assign' id:24600 op device:{requested: '', assigned: ''} def:{{{node lstm_110/lstm_cell_110/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_110/lstm_cell_110/recurrent_kernel, lstm_110/lstm_cell_110/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:02:28.569390: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_108/lstm_cell_108/bias/v/Assign' id:26593 op device:{requested: '', assigned: ''} def:{{{node lstm_108/lstm_cell_108/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_108/lstm_cell_108/bias/v, lstm_108/lstm_cell

Test MAE:
1.3347925819732525
[1.12646723 1.44146846]
1.2839678424635743


In [6]:
ra = 0.2

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35369


In [7]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_33.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:02:51.518103: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_15/kernel/Assign' id:34052 op device:{requested: '', assigned: ''} def:{{{node dense_15/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_15/kernel, dense_15/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:02:53.412985: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_15/bias/m/Assign' id:34656 op device:{requested: '', assigned: ''} def:{{{node conv2d_15/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_15/bias/m, conv2d_15/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and wi

Test MAE:
1.283711926682907
[1.15467864 1.33831536]
1.2464969990884556


In [8]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_34.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:03:08.057983: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_183/lstm_cell_183/bias/Assign' id:42911 op device:{requested: '', assigned: ''} def:{{{node lstm_183/lstm_cell_183/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_183/lstm_cell_183/bias, lstm_183/lstm_cell_183/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:03:10.354984: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_166/lstm_cell_166/kernel/v/Assign' id:44805 op device:{requested: '', assigned: ''} def:{{{node lstm_166/lstm_cell_166/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_166/lstm_cell_166/kernel/v, lstm_166/lstm_cell_166/kernel/v/Initializer/zeros)}}' was c

Test MAE:
1.2408769020408976
[1.12746766 1.28529894]
1.2063832993016523


In [9]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_35.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:03:25.881840: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_192/lstm_cell_192/recurrent_kernel/Assign' id:47651 op device:{requested: '', assigned: ''} def:{{{node lstm_192/lstm_cell_192/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_192/lstm_cell_192/recurrent_kernel, lstm_192/lstm_cell_192/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:03:28.481151: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_212/lstm_cell_212/kernel/v/Assign' id:54171 op device:{requested: '', assigned: ''} def:{{{node lstm_212/lstm_cell_212/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_212/lstm_cell_212/kernel/v, lstm_212/lst

Test MAE:
1.258198283758818
[1.12797295 1.29984168]
1.213907316327095


In [10]:
ra = 0.3

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35573


In [11]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_36.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:03:50.208310: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_242/lstm_cell_242/kernel/Assign' id:58942 op device:{requested: '', assigned: ''} def:{{{node lstm_242/lstm_cell_242/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_242/lstm_cell_242/kernel, lstm_242/lstm_cell_242/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:03:53.265670: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_25/kernel/v/Assign' id:62967 op device:{requested: '', assigned: ''} def:{{{node conv2d_25/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_25/kernel/v, conv2d_25/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it

Test MAE:
1.1282494317750638
[0.98649849 1.15164126]
1.0690698743868274


In [12]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_37.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:04:09.042752: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_292/lstm_cell_292/recurrent_kernel/Assign' id:70275 op device:{requested: '', assigned: ''} def:{{{node lstm_292/lstm_cell_292/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_292/lstm_cell_292/recurrent_kernel, lstm_292/lstm_cell_292/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:04:12.471073: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_271/lstm_cell_271/kernel/v/Assign' id:72408 op device:{requested: '', assigned: ''} def:{{{node lstm_271/lstm_cell_271/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_271/lstm_cell_271/kernel/v, lstm_271/lst

Test MAE:
1.161484427478801
[1.00544137 1.20133954]
1.103390454244213


In [13]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_38.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:04:29.134153: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_300/lstm_cell_300/bias/Assign' id:74873 op device:{requested: '', assigned: ''} def:{{{node lstm_300/lstm_cell_300/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_300/lstm_cell_300/bias, lstm_300/lstm_cell_300/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:04:33.000827: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_308/lstm_cell_308/recurrent_kernel/v/Assign' id:81644 op device:{requested: '', assigned: ''} def:{{{node lstm_308/lstm_cell_308/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_308/lstm_cell_308/recurrent_kernel/v, lstm_308/lstm_cell_308/recurr

Test MAE:
1.1767450020092876
[1.00766316 1.24794236]
1.1278027596593905


In [14]:
ra = 0.4

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35777


In [15]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_39.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:04:55.982287: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_36/bias/Assign' id:83140 op device:{requested: '', assigned: ''} def:{{{node conv2d_36/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_36/bias, conv2d_36/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:05:00.602904: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_360/lstm_cell_360/bias/m/Assign' id:90462 op device:{requested: '', assigned: ''} def:{{{node lstm_360/lstm_cell_360/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_360/lstm_cell_360/bias/m, lstm_360/lstm_cell_360/bias/m/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. Th

Test MAE:
1.121896776595926
[1.04513398 1.06396126]
1.0545476195858974


In [16]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_40.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:05:17.267992: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_371/lstm_cell_371/kernel/Assign' id:92826 op device:{requested: '', assigned: ''} def:{{{node lstm_371/lstm_cell_371/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_371/lstm_cell_371/kernel, lstm_371/lstm_cell_371/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:05:21.887415: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_42/kernel/m/Assign' id:99258 op device:{requested: '', assigned: ''} def:{{{node conv2d_42/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_42/kernel/m, conv2d_42/kernel/m/Initializer/zeros)}}' was changed by setting attribute after it

Test MAE:
1.155457592185806
[1.09940953 1.0851    ]
1.0922547648934757


In [17]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_41.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:05:39.655754: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_409/lstm_cell_409/recurrent_kernel/Assign' id:102237 op device:{requested: '', assigned: ''} def:{{{node lstm_409/lstm_cell_409/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_409/lstm_cell_409/recurrent_kernel, lstm_409/lstm_cell_409/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:05:44.720947: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_410/lstm_cell_410/recurrent_kernel/v/Assign' id:109202 op device:{requested: '', assigned: ''} def:{{{node lstm_410/lstm_cell_410/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_410/lstm_cell_410/

Test MAE:
1.1220987269691391
[1.07166133 1.04965927]
1.0606603026390076


In [18]:
ra = 0.5

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

35981


In [19]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_42.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:06:08.791024: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_470/lstm_cell_470/recurrent_kernel/Assign' id:115310 op device:{requested: '', assigned: ''} def:{{{node lstm_470/lstm_cell_470/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_470/lstm_cell_470/recurrent_kernel, lstm_470/lstm_cell_470/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:06:14.307794: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_451/lstm_cell_451/bias/m/Assign' id:117855 op device:{requested: '', assigned: ''} def:{{{node lstm_451/lstm_cell_451/bias/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_451/lstm_cell_451/bias/m, lstm_451/lstm_ce

Test MAE:
1.125011901294484
[1.02143679 1.10979055]
1.0656136695076437


In [20]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_43.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:06:32.880736: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_508/lstm_cell_508/kernel/Assign' id:124681 op device:{requested: '', assigned: ''} def:{{{node lstm_508/lstm_cell_508/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_508/lstm_cell_508/kernel, lstm_508/lstm_cell_508/kernel/Initializer/random_uniform)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:06:38.778242: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_484/lstm_cell_484/bias/v/Assign' id:127669 op device:{requested: '', assigned: ''} def:{{{node lstm_484/lstm_cell_484/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_484/lstm_cell_484/bias/v, lstm_484/lstm_cell_484/bias/v/Initializer/zeros

Test MAE:
1.108341235740512
[1.00896213 1.09902473]
1.0539934298571418


In [21]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_44.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:06:57.474026: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_526/lstm_cell_526/bias/Assign' id:130899 op device:{requested: '', assigned: ''} def:{{{node lstm_526/lstm_cell_526/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_526/lstm_cell_526/bias, lstm_526/lstm_cell_526/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:07:03.720922: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_537/lstm_cell_537/recurrent_kernel/v/Assign' id:137135 op device:{requested: '', assigned: ''} def:{{{node lstm_537/lstm_cell_537/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_537/lstm_cell_537/recurrent_kernel/v, lstm_537/lstm_cell_537/recu

Test MAE:
1.1552918992790522
[1.07836395 1.11894989]
1.098656920825734


In [22]:
ra = 0.6

flag_tr_val_te = split_data_few_shot(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, ratio = ra, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature_combine, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

36185


In [23]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_45.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:07:29.061518: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_576/lstm_cell_576/recurrent_kernel/Assign' id:142201 op device:{requested: '', assigned: ''} def:{{{node lstm_576/lstm_cell_576/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_576/lstm_cell_576/recurrent_kernel, lstm_576/lstm_cell_576/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:07:35.726680: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_587/lstm_cell_587/bias/v/Assign' id:146566 op device:{requested: '', assigned: ''} def:{{{node lstm_587/lstm_cell_587/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_587/lstm_cell_587/bias/v, lstm_587/lstm_ce

Test MAE:
1.0074420733486904
[0.87542173 1.03346606]
0.9544438965180341


In [24]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_46.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:07:55.896649: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_593/lstm_cell_593/recurrent_kernel/Assign' id:148232 op device:{requested: '', assigned: ''} def:{{{node lstm_593/lstm_cell_593/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_593/lstm_cell_593/recurrent_kernel, lstm_593/lstm_cell_593/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:08:03.525577: W tensorflow/c/c_api.cc:304] Operation '{name:'conv2d_64/bias/v/Assign' id:155270 op device:{requested: '', assigned: ''} def:{{{node conv2d_64/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_64/bias/v, conv2d_64/bias/v/Initializer/zeros)}}' was changed by s

Test MAE:
1.0287708065965597
[0.90412667 1.02654669]
0.9653366786592147


In [25]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_few_shot_p"+str(test_person_id[0])+'_47.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:08:23.902595: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_663/lstm_cell_663/bias/Assign' id:162754 op device:{requested: '', assigned: ''} def:{{{node lstm_663/lstm_cell_663/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_663/lstm_cell_663/bias, lstm_663/lstm_cell_663/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:08:31.385129: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_647/lstm_cell_647/kernel/m/Assign' id:164165 op device:{requested: '', assigned: ''} def:{{{node lstm_647/lstm_cell_647/kernel/m/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_647/lstm_cell_647/kernel/m, lstm_647/lstm_cell_647/kernel/m/Initializer/zeros)}}' was

Test MAE:
0.9800572786845413
[0.83817007 1.00784644]
0.9230082578518811


In [26]:
from feature_emotion import feature_extract, split_data, label_unique_tuples, split_data_unknown
gts, sensor_nums, walk_nums, trace_nums, people_nums, spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features = feature_extract(person_nums)

walk_nums_all = np.squeeze(walk_nums)
trace_nums_all = np.squeeze(trace_nums)
people_nums_all = np.squeeze(people_nums)

flag_tr_val_te = split_data_unknown(test_person_id, walk_nums_all, trace_nums_all, people_nums_all, person_nums, rand_seed = 42)
## Data Normalization before training ans testing
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, concatenate, LSTM
from sklearn.preprocessing import StandardScaler
scalers = []
X_train_normalized = []
X_val_normalized = []
X_test_normalized = []
train_idx = np.where(flag_tr_val_te ==0)[0]
np.random.shuffle(train_idx)
val_idx = np.where(flag_tr_val_te ==1)[0]
test_idx = np.where(flag_tr_val_te ==2)[0]

for i, feature in enumerate([spe_centr, delta_spe_centr, spe_crest, delta_spe_crest, spe_decrease, delta_spe_decrease, spe_entropy, delta_spe_entropy, spe_flatness, delta_spe_flatness, spe_flux, delta_spe_flux, spe_kurtosis, delta_spe_kurtosis, spe_skewness, delta_spe_skewness, spe_rfp, delta_spe_rfp, spe_slope, delta_spe_slope, spe_spread, delta_spe_spread, wlk_fre, wlk_fres_trace, cwt_figs_all, cwt_sum_all_0, cwt_sum_all_1, cwt_sum_all_2, cwt_sum_all_3, high_fre_compos, pitchs, low_fre_compos, auto_corrs, real_hils, imag_hils, dur_time_1_alls, dur_time_2_alls, jitters, shimmers, jitter_rap, hrs, feature, slope, zcrs, fft_features, energy_alls, log_energy_alls, smoothe_energy_alls, legendres, double_support_time, pdps_new, lpcs, ceps_features]):
    scaler = StandardScaler()
    if len(feature.shape)==2:
        X_train_i = feature[train_idx,:]
        X_val_i = feature[val_idx,:]
        X_test_i = feature[test_idx,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i)
        X_val_normalized_i = scaler.transform(X_val_i)
        X_test_normalized_i = scaler.transform(X_test_i)
        scalers.append(scaler)
    else:
        X_train_i = feature[train_idx,:,:]
        X_val_i = feature[val_idx,:,:]
        X_test_i = feature[test_idx,:,:]
        X_train_normalized_i = scaler.fit_transform(X_train_i.reshape(X_train_i.shape[0], -1)).reshape(X_train_i.shape)
        X_val_normalized_i = scaler.transform(X_val_i.reshape(X_val_i.shape[0], -1)).reshape(X_val_i.shape)
        X_test_normalized_i = scaler.transform(X_test_i.reshape(X_test_i.shape[0], -1)).reshape(X_test_i.shape)
        scalers.append(scaler)
    X_train_normalized.append(X_train_normalized_i)
    X_val_normalized.append(X_val_normalized_i)
    X_test_normalized.append(X_test_normalized_i)
y_train = gts[train_idx,:]
y_val = gts[val_idx,:]
y_test = gts[test_idx,:]
X_train_normalized_new = []
combined_feature = np.empty((len(X_train_normalized[0]),0))
for feature in X_train_normalized:
    if len(feature.shape) == 3:
        X_train_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_train_normalized_new.append(feature)
X_train_normalized_new.append(combined_feature)

X_val_normalized_new = []
combined_feature = np.empty((len(X_val_normalized[0]),0))
for feature in X_val_normalized:
    if len(feature.shape) == 3:
        X_val_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_val_normalized_new.append(feature)
X_val_normalized_new.append(combined_feature)

X_test_normalized_new = []
combined_feature = np.empty((len(X_test_normalized[0]),0))
for feature in X_test_normalized:
    if len(feature.shape) == 3:
        X_test_normalized_new.append(feature)
    elif feature.shape[1] <20:
        combined_feature = np.hstack((combined_feature, feature))
    else:
        X_test_normalized_new.append(feature)
X_test_normalized_new.append(combined_feature)

34973


In [27]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_20.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:09:43.610002: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_15_1/lstm_cell_681/bias/Assign' id:168943 op device:{requested: '', assigned: ''} def:{{{node lstm_15_1/lstm_cell_681/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_15_1/lstm_cell_681/bias, lstm_15_1/lstm_cell_681/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:09:51.499283: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_6_1/lstm_cell_672/recurrent_kernel/v/Assign' id:173864 op device:{requested: '', assigned: ''} def:{{{node lstm_6_1/lstm_cell_672/recurrent_kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_6_1/lstm_cell_672/recurrent_kernel/v, lstm_6_1/lstm_cell_672/

Test MAE:
1.635733475581664
[1.28272678 1.9336176 ]
1.6081721923760408


In [28]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_21.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:10:19.926258: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_8_2/lstm_cell_711/bias/Assign' id:177054 op device:{requested: '', assigned: ''} def:{{{node lstm_8_2/lstm_cell_711/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_8_2/lstm_cell_711/bias, lstm_8_2/lstm_cell_711/bias/Initializer/concat)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:10:28.211740: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_18_2/lstm_cell_721/kernel/v/Assign' id:183270 op device:{requested: '', assigned: ''} def:{{{node lstm_18_2/lstm_cell_721/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_18_2/lstm_cell_721/kernel/v, lstm_18_2/lstm_cell_721/kernel/v/Initializer/zeros)}}'

Test MAE:
1.5601855418268276
[1.2850528  1.76940243]
1.5272276175798043


In [29]:
## Evaluation on test data
import tensorflow as tf
m_name = "./checkpoints/unknown_person_p"+str(test_person_id[0])+'_22.h5'
model = tf.keras.models.load_model(m_name)
from feature_emotion import feature_extract, split_data, label_unique_tuples
trace_wlk_num = label_unique_tuples(people_nums_all, walk_nums_all, trace_nums_all)
trace_wlk_num = np.array(trace_wlk_num )

y_pred = model.predict(X_test_normalized_new)
print('Test MAE:')
err = np.mean(np.abs(y_pred - y_test))
print(err)
## Evaluation: trace median vote
trace_num_test = trace_wlk_num[test_idx]
u = np.unique(trace_num_test)
pred_trace = np.empty((0, 2))
gt_trace = np.empty((0, 2))
for i in u:
  trace_idx = np.where(trace_num_test == i)[0]
  y_pred_trace = y_pred[trace_idx,:]
  pred = np.mean(y_pred_trace, axis = 0)
  pred_trace = np.vstack((pred_trace, pred))
  gt_t = y_test[trace_idx[0],:]
  gt_trace = np.vstack((gt_trace, gt_t))
print(np.mean(np.abs(pred_trace-gt_trace), axis = 0))
print(np.mean(np.abs(pred_trace-gt_trace)))

2023-12-04 19:10:57.990255: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_27_3/lstm_cell_767/recurrent_kernel/Assign' id:189318 op device:{requested: '', assigned: ''} def:{{{node lstm_27_3/lstm_cell_767/recurrent_kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_27_3/lstm_cell_767/recurrent_kernel, lstm_27_3/lstm_cell_767/recurrent_kernel/Initializer/mul_1)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-12-04 19:11:06.692131: W tensorflow/c/c_api.cc:304] Operation '{name:'lstm_15_3/lstm_cell_755/bias/v/Assign' id:192466 op device:{requested: '', assigned: ''} def:{{{node lstm_15_3/lstm_cell_755/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](lstm_15_3/lstm_cell_755/bias/v, lstm_15_3

Test MAE:
1.6674748813141969
[1.31013138 1.95450966]
1.6323205211458827
